In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab as plt
import seaborn as sb
from pylab import rcParams
import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn import datasets
from influxdb import InfluxDBClient

In [46]:
df1=pd.read_csv("C:/Users/Swarupa/Desktop/Terrorism.csv")

C:\Users\Swarupa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
df=df1[['eventid','iyear','country_txt']]

In [48]:
df.dtypes

eventid         int64
iyear           int64
country_txt    object
dtype: object

In [49]:
df.head()

,eventid,iyear,country_txt
0,197000000001,1970,Dominican Republic
1,197000000002,1970,Mexico
2,197001000001,1970,Philippines
3,197001000002,1970,Greece
4,197001000003,1970,Japan


In [50]:
grouped=df.groupby(["iyear","country_txt"])["eventid"].count().reset_index(name="Number")

In [51]:
grouped.head()

,iyear,country_txt,Number
0,1970,Argentina,21
1,1970,Australia,1
2,1970,Belgium,1
3,1970,Bolivia,1
4,1970,Brazil,6


In [52]:
grouped['Number']=(grouped['Number']/grouped['Number'].max())*100

In [53]:
grouped.head()

,iyear,country_txt,Number
0,1970,Argentina,0.534896
1,1970,Australia,0.025471
2,1970,Belgium,0.025471
3,1970,Bolivia,0.025471
4,1970,Brazil,0.152827


In [54]:
grouped.to_csv("Terrorism_normalized.csv",index=False)

In [55]:
grouped.head()

,iyear,country_txt,Number
0,1970,Argentina,0.534896
1,1970,Australia,0.025471
2,1970,Belgium,0.025471
3,1970,Bolivia,0.025471
4,1970,Brazil,0.152827


In [56]:
def read_data():
    with open('C:/Users/Swarupa/Desktop/Terrorism_normalized.csv') as f: #Input the name of your csv file here
        return [x.split(',') for x in f.readlines()[1:]]
def read_header():
    with open('C:/Users/Swarupa/Desktop/Terrorism_normalized.csv') as f: #Input the name of your csv file here
        return [x.split(',') for x in f.readlines()[:1]]

def main(host='localhost', port=8086):       #input your port number which has been set for influx
    """Instantiate the connection to the InfluxDB client."""
    user = 'Swarupa'
    password = ''
    dbname = 'SocialWeather'
    # Temporarily avoid line protocol time conversion issues #412, #426, #431.
    protocol = 'json'
    client = InfluxDBClient(host, port, user, password, dbname)
    
    print("Create DataFrame");
    b=read_header();
    length=len(b);
    print(b);
    narr=(b[-1])
    narr[-1]=narr[-1].split('\n',1)[0]
    print(narr); # getting /n (new line?,why?)
    a = read_data();
    for metric in a:
        for i in range(len(narr)):
            influx_metric = [{
                'measurement': 'Terrorism_normalized',
                'tags': {
                    'data_normalized?': 'normalized'                 },
                'time':str(metric[0])+"-12-31"+"T00:00:00Z",
                'fields': {
                    narr[i]: metric[i].split('\n',1)[0]

                }
            }]
            client.write_points(influx_metric)

    
if __name__ == '__main__':
    main()

Create DataFrame
[['iyear', 'country_txt', 'Number\n']]
['iyear', 'country_txt', 'Number']
